In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
def extract_details(url, column="Szerződők"):
    return pd.read_html(url)[0].set_index(0).transpose()[column][1]


def get_participants_df(res):
    links = BeautifulSoup(res.content, 'html.parser').find('table', class_="results_with-link").find_all('tr')

    subpages = []
    for row in links[1:]:
        link = row.find('a')
        id_ = link.get_text()
        url = 'https://kereso-core.kozbeszerzes.hu' + link.get('href')
        try:
            participants = extract_details(url, "Szerződők")
        except:
            print(f'FAILED participant info: {id_}')
            participants = "ISMERETLEN"
        subpages.append({'azon': id_, 'Szerződők': participants})

    return pd.DataFrame(subpages)


def get_main_df(res):
    return pd.read_html(res.content)[0].rename(columns={'Eljárás azonosítója': 'azon'})


def generate_df(res):
    main_df = get_main_df(res)
    participants_df = get_participants_df(res)
    return main_df.merge(participants_df, on='azon')


def scrape_page(page_id):
    res = requests.get(f"https://kereso-core.kozbeszerzes.hu/lista/kozbeszerzes/?oldal={page_id}")
    return generate_df(res)

In [ ]:
tables = []
for page_id in tqdm(range(1, 501)):
    try:
        scraped = scrape_page(page_id)
        tables.append(scrape_page(page_id))
    except Exception as e:
        print(f'FAILED: {page_id} with error: {e}')
    
result = pd.concat(tables)

In [ ]:
result.to_excel('data/nyertes_kozbesz_2021-03-23--2023.xlsx')

In [ ]:
url = "https://kereso-core.kozbeszerzes.hu/kereses/kozbeszerzes/osszetett/?q=&alairas_datuma_0=1990-01-01&alairas_datuma_1=2020-03-24#talalatok"

---

In [ ]:
def extract_articles(res):
    articles = BeautifulSoup(res.content).find('div', class_="article__container").find_all('div', class_="roll")
    results = []
    for article in articles:
        try:
            article_link = article.find('a', class_="roll__title")
            article_url = "https://kereso-core.kozbeszerzes.hu" + article_link.get('href')
            article_details = article.find('p', class_='roll__details')
            result = {
                'Cím': article_link.get_text().strip(),
                "Ajánlatkérő neve": extract_details(article_url, 'Ajánlatkérő neve:'),
                'Szerződők': extract_details(article_url, 'Szerződők'),
                'Szerződés tárgya': extract_details(article_url, "Szerződés tárgya:"),
                'Szerződés megkötéskori értéke': extract_details(article_url, "Szerződés megkötéskori értéke"),
                'Típus': article_details.find('span', class_='roll__type').get_text().strip(),
                'Szereződés előzménye': article_details.find('span', class_='roll__antecedent').get_text().strip(),
                'Szereződés jellege': article_details.find('span', class_='roll__nature').get_text().strip(),
                'Szereződés státusza': article_details.find('span', class_='roll__state').get_text().strip(),
                'Lejárat módja': article_details.find('span', class_='roll__expiry').get_text().strip(),
            }
            
        except Exception as e:
            print(f'FAILED to extract: {article}')
        else:
            results.append(result)
            
    return results

In [ ]:
independents = []

for page_id in tqdm(range(1, 275)):
    try:
        res = requests.get(f"https://kereso-core.kozbeszerzes.hu/kereses/eljarasfuggetlen/egyszeru/?q=&oldal={page_id}")
        articles = extract_articles(res)
        independents.extend(articles)
    except Exception as e:
        print(f'FAILED: {page_id} with error: {e}')

independents_df = pd.DataFrame(independents)
independents_df.to_excel('data/kozbesz_eljaras_nelkul.xlsx')